In [1]:
!pip install ultralytics opencv-python


In [1]:
!pip install numpy==1.24.4 --force-reinstall


  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "C:\Users\vshukla\AppData\Local\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\vshukla\AppData\Local\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\vshukla\AppData\Local\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "C:\Users\vshukla\AppData\Local\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backend
     

In [4]:
from ultralytics import YOLO
import cv2



In [3]:
with open("datasets/face_mask_kaggle/data.yaml", "r") as file:
    print(file.read())


path: datasets/face_mask_kaggle
train: images/train
val: images/val

names:
  0: with_mask
  1: mask_weared_incorrect
  2: without_mask



In [6]:
# Load trained model
model = YOLO(r"C:\Users\vshukla\runs\detect\train5\weights\best.pt")

# Open webcam
cap = cv2.VideoCapture(0)  # Change index if you have multiple cameras

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO inferencea
    results = model(frame, imgsz=640, verbose=False)[0]

    # Draw results on frame
    annotated_frame = results.plot()

    # Show the frame
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [2]:
#to know how many are from with_mask = '0' and without_mask = '1'

from collections import Counter
import os

label_folder = "datasets/face_mask_kaggle/labels/train"
class_counts = Counter()

for file in os.listdir(label_folder):
    with open(os.path.join(label_folder, file), 'r') as f:
        for line in f:
            class_id = line.strip().split()[0]
            class_counts[class_id] += 1

print("Class Counts:", class_counts)


Class Counts: Counter({'0': 3125, '2': 692, '1': 117})


In [4]:
import os, shutil
import random

IMG_DIR = "datasets/face_mask_kaggle/images"
LBL_DIR = "datasets/face_mask_kaggle/labels"

out_img = "datasets/face_mask_kaggle/images"
out_lbl = "datasets/face_mask_kaggle/labels"

train_img, val_img = f"{out_img}/train", f"{out_img}/val"
train_lbl, val_lbl = f"{out_lbl}/train", f"{out_lbl}/val"

for p in [train_img, val_img, train_lbl, val_lbl]:
    os.makedirs(p, exist_ok=True)

images = [f for f in os.listdir(IMG_DIR) if f.endswith(".png") or f.endswith(".jpg")]
random.shuffle(images)
split = int(0.8 * len(images))

for i, img in enumerate(images):
    label_file = img.replace(".png", ".txt").replace(".jpg", ".txt")
    if i < split:
        shutil.copy(os.path.join(IMG_DIR, img), os.path.join(train_img, img))
        shutil.copy(os.path.join(LBL_DIR, label_file), os.path.join(train_lbl, label_file))
    else:
        shutil.copy(os.path.join(IMG_DIR, img), os.path.join(val_img, img))
        shutil.copy(os.path.join(LBL_DIR, label_file), os.path.join(val_lbl, label_file))


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="datasets/face_mask_kaggle/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16
)


Ultralytics 8.3.166  Python-3.10.18 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i7-1365U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/face_mask_kaggle/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=1

train: Scanning C:\Users\vshukla\Downloads\Face-mask-detection\datasets\face_mask_kaggle\labels\train.cache... 826 imag

val: Fast image access  (ping: 1.90.3 ms, read: 93.743.3 MB/s, size: 375.0 KB)



C:\Users\vshukla\AppData\Local\anaconda3\envs\yolov8-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\vshukla\Downloads\Face-mask-detection\datasets\face_mask_kaggle\labels\val.cache... 315 images, 
C:\Users\vshukla\AppData\Local\anaconda3\envs\yolov8-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to C:\Users\vshukla\runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\vshukla\runs\detect\train5
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        315       1514      0.932      0.886      0.938      0.681



30 epochs completed in 6.408 hours.
Optimizer stripped from C:\Users\vshukla\runs\detect\train5\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\vshukla\runs\detect\train5\weights\best.pt, 6.2MB

Validating C:\Users\vshukla\runs\detect\train5\weights\best.pt...
Ultralytics 8.3.166  Python-3.10.18 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i7-1365U)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [01:02


                   all        315       1514      0.924      0.887       0.94      0.682
             with_mask        279       1205      0.943      0.941      0.976      0.725
 mask_weared_incorrect         43         54      0.921      0.833      0.906       0.66
          without_mask        112        255      0.908      0.886      0.938      0.661
Speed: 3.1ms preprocess, 141.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to C:\Users\vshukla\runs\detect\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001ADBF4B4850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          